In [ ]:
import seaborn as sns  
import matplotlib.pyplot as plt  
import pandas as pd 
from tqdm import tqdm  
import os 
import statsmodels.api as sm  
import numpy as np

In [ ]:
folder_path = '/home2/zhenglijing/work/bigwig/276depth>=10.csv'
dfall=pd.read_csv(folder_path,index_col=0)
dfall

In [ ]:
path='/home2/zhenglijing/work_0511/sub.csv'
df=pd.read_csv(path)
df

In [ ]:
normal_numbers = df[df['type'] == 'normal']['number'].tolist()

In [ ]:
dfall=dfall[~dfall['number'].isin(normal_numbers)]

In [ ]:
listtarget=(dfall['number'].unique()).tolist()

In [ ]:
len(listtarget)

In [ ]:
new_df = dfall[['combine', 'vaf','Gene.refGene',]]

In [ ]:
new_df

In [ ]:
new_df['combine']=new_df['combine']+'-'+new_df['Gene.refGene'].astype(str)

In [ ]:
chip_df=new_df

In [ ]:
new_df.head()

In [ ]:
unique_count = new_df['combine'].nunique()
print(f"combine列中不重复的行数: {unique_count}")

## 制作nochipdf

In [ ]:
folder_path1 = '/home2/zhenglijing/work/SNP_bcf/dele_AD/public_data/annovar_annote_addpvaf_sup/'  
folder_path2= '/home2/zhenglijing/work/SNP_bcf/dele_AD/public_data/annovar_annote_addpvaf/'  
folder_path3 = '/home2/zhenglijing/work/annovar/deleAD/table_annot_addpvaf/'  
fileall=[]
for file in os.listdir(folder_path1):
    df=pd.read_csv(folder_path1+file)
    fileall.append(df)
for file in os.listdir(folder_path2):
    df=pd.read_csv(folder_path2+file)
    fileall.append(df)
for file in os.listdir(folder_path3):
    df=pd.read_csv(folder_path3+file)
    fileall.append(df)


In [ ]:
dfall_allsystem=pd.concat(fileall)

In [ ]:
path='/home2/zhenglijing/work_0511/sub.csv'
df=pd.read_csv(path)
normal_numbers = df[df['type'] == 'normal']['number'].tolist()
dfall_allsystem=dfall_allsystem[~dfall_allsystem['number'].isin(normal_numbers)]

In [ ]:
dfall_allsystem.head()

In [ ]:
folder_path = '/home2/zhenglijing/work/SNP_bcf/dele_AD/public_data/420finalresult'
all_data_df = pd.DataFrame()

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    if filename.endswith('.csv'):  
        df = pd.read_csv(file_path) 
        all_data_df = pd.concat([all_data_df, df], ignore_index=True)
all_data_df.head()

In [ ]:
path='/home2/zhenglijing/work_0511/sub.csv'
df=pd.read_csv(path)
normal_numbers = df[df['type'] == 'normal']['number'].tolist()
all_data_df=all_data_df[~all_data_df['number'].isin(normal_numbers)]

In [ ]:
all_data_df.head()

In [ ]:
number_typedf=all_data_df[['number','type']].copy()

In [ ]:
number_typedf=number_typedf.drop_duplicates(subset=['number'])

In [ ]:
number_typedf.to_csv('number_type_375')

In [ ]:
number_typedf

In [ ]:
number_to_type = dict(zip(number_typedf['number'], number_typedf['type']))

dfall_allsystem['type'] = dfall_allsystem['number'].map(number_to_type)

In [ ]:
dfall_allsystem

In [ ]:
exclude_types = ['mother', 'father', 'normal', 'nomal']

dfall_allsystem = dfall_allsystem[~dfall_allsystem['type'].isin(exclude_types)]
len(dfall_allsystem['number'].unique())

In [ ]:
exclude_types =['ERR3366532','ERR3366542']

dfall_allsystem = dfall_allsystem[~dfall_allsystem['number'].isin(exclude_types)]
len(dfall_allsystem['number'].unique())

In [ ]:
dfall_allsystem.head()

In [ ]:
combined_df=dfall_allsystem

In [ ]:
1

In [ ]:
 combined_df.columns

In [ ]:
combined_df.loc[combined_df['ExAC_EAS'] == '.', 'ExAC_EAS'] = 0

In [ ]:
combined_df.loc[ combined_df['EAS.sites.2015_08']=='.' , 'EAS.sites.2015_08' ]=0


In [ ]:
df_fil=combined_df[(combined_df['EAS.sites.2015_08'].astype(float) <0.001)&\
            (combined_df['ExAC_EAS'].astype(float)  <0.001)&
            (combined_df['p_value']<0.001)&(combined_df['vaf']<0.5)]
df_fil

In [ ]:
df_fil.head()

In [ ]:
df_fil['combine']=df_fil['Chr']+'-'+df_fil['Start'].astype(str)+'-'+df_fil['Ref']\
+'-'+df_fil['Alt']

In [ ]:
df=df_fil[(df_fil['Gene.refGene'].str.contains(';'))&(~df_fil['Gene.refGene'].str.contains('NONE'))]\
[['Gene.refGene','vaf','combine']]
df_expanded = pd.DataFrame(columns=df.columns)
df_current_expanded = pd.DataFrame(columns=df.columns)
for index, row in tqdm(df.iterrows()):
    if ';' in row['Gene.refGene']:
        genes = row['Gene.refGene'].split(';')        new_rows = pd.DataFrame([{'Gene.refGene': gene, **{col: row[col] for col in df.columns if col != 'Gene.refGene'}} for gene in genes])  
        df_current_expanded = pd.concat([df_current_expanded, new_rows], ignore_index=True)  
    else:  
        df_current_expanded = pd.concat([df_current_expanded, pd.DataFrame([row])], ignore_index=True)
df_expanded = pd.concat([df_expanded, df_current_expanded], ignore_index=True)

df_expanded.head()

In [ ]:
df_expanded=pd.read_csv('/home2/zhenglijing/work_0511/df_expanded.csv') 

In [ ]:
df_expanded.head()

In [ ]:
unsplit=df_fil[~df_fil['Gene.refGene'].str.contains(';').fillna(True)][['Gene.refGene','vaf','combine']]
unsplit['combine']=unsplit['combine']+'-'+unsplit['Gene.refGene'].astype(str)
unsplit['combine']=unsplit['combine']+'-'+unsplit['Gene.refGene'].astype(str)
unsplit

In [ ]:
df_expanded['combine']=df_expanded['combine']+'-'+df_expanded['Gene.refGene'].astype(str)
df_expanded['combine']=df_expanded['combine']+'-'+df_expanded['Gene.refGene'].astype(str)
df_expanded

In [ ]:
nochip_df = pd.concat([df_expanded,unsplit])[['combine', 'vaf']]

In [ ]:
unique_count = nochip_df['combine'].nunique()
print(f"combine列中不重复的行数: {unique_count}")

In [ ]:
nochip_df.head()

In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


nochip_unique = nochip_df[~nochip_df['combine'].isin(chip_df['combine'])]
vaf_nochip = nochip_unique['vaf'].values
vaf_chip = chip_df['vaf'].values

t_stat, p_value = stats.ttest_ind(vaf_nochip, vaf_chip)
print(f'T-statistic: {t_stat}, P-value: {p_value}')


In [ ]:
vaf_nochip.shape

In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


nochip_unique = nochip_df[~nochip_df['combine'].isin(chip_df['combine'])]
vaf_nochip = nochip_unique['vaf'].values
vaf_chip = chip_df['vaf'].values

t_stat, p_value = stats.mannwhitneyu(vaf_nochip, vaf_chip,alternative='less')
print(f'T-statistic: {t_stat}, P-value: {p_value}')


In [ ]:
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


nochip_unique = nochip_df[~nochip_df['combine'].isin(chip_df['combine'])]
vaf_nochip = nochip_unique['vaf'].values
vaf_chip = chip_df['vaf'].values

t_stat, p_value = stats.ttest_ind(vaf_nochip, vaf_chip)
print(f'T-statistic: {t_stat}, P-value: {p_value}')


In [ ]:
ax=sns.boxplot(x='source', y='vaf', data=combined_df,width=0.7,
            palette=['#b3b3b3','#fb8073'],)

In [ ]:
from statannotations.Annotator import Annotator

In [ ]:
plt.rcParams["font.size"] = 12
plt.rcParams["font.family"] = "Arial"
plt.rcParams["pdf.fonttype"] = 42

In [ ]:
import random

In [ ]:
nochip_df=nochip_df.reset_index()

In [ ]:
rd_index=random.sample(list(nochip_df.index),len(chip_df))

In [ ]:
chip_df

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

combined_df = pd.concat([
    nochip_df[nochip_df.index.isin(rd_index)].assign(source='NOCHIPI'),    chip_df.assign(source='CHIPI')], ignore_index=True)

plt.rcParams['font.family'] = 'Arial'plt.figure(figsize=(2, 3),dpi=500)
ax=sns.boxplot(x='source', y='vaf', data=combined_df,width=0.7,
            palette=['#b3b3b3','#fb8073',],
           )plt.ylim(bottom=0, top=0.4)

pairs=[('NOCHIPI', 'CHIPI'),]
annotator = Annotator(ax, pairs, data=combined_df, 
                      x='source', y='vaf',)
annotator.configure(test='Mann-Whitney', text_format='star', loc='outside')
annotator.apply_and_annotate()
plt.tick_params(which='both', width=1.5, length=7, left=True, bottom=False)
plt.title('')
plt.xticks(fontsize=12)
plt.xlabel('')
plt.yticks([0,0.2,0.4])
plt.ylabel('VAF',fontsize=12)
sns.despine(bottom=True,left=False)
plt.savefig('vaf_chipi_nochip.pdf')
plt.show()

In [ ]:
from scipy import stats

In [ ]:
stats.mannwhitneyu(nochip_df['vaf'],chip_df['vaf'],alternative='less')

In [ ]:
stats.mannwhitneyu(nochip_df[nochip_df.index.isin(rd_index)]['vaf'],chip_df['vaf'],alternative='less')